In [ ]:
pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp37-none-any.whl size=19877 sha256=906a1eb3dde5969b3da614f0bdc71abf84fb0348691359f4580b698cd89b074b
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from vecstack import stacking
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split, KFold, RepeatedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
features = pd.read_csv(r"/content/drive/MyDrive/DS Project/processed features - Complete.csv", delimiter=";" ,names=["Label", "Number of Samples", "Min", "Max", "Median", "Mean", "SD"])

In [ ]:
print(features.head())
print(features.info())

   Label  Number of Samples    Min    Max  Median        Mean         SD
0    1.0               79.0  305.0  520.0   375.0  378.544304  30.514075
1    1.0               83.0  285.0  390.0   365.0  360.662651  16.911732
2    1.0               83.0  200.0  525.0   360.0  363.012048  30.615570
3    1.0               84.0  335.0  365.0   355.0  356.666667   5.634362
4    1.0               85.0  340.0  365.0   355.0  354.588235   5.041695
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27516 entries, 0 to 27515
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Label              27516 non-null  float64
 1   Number of Samples  27516 non-null  float64
 2   Min                27516 non-null  float64
 3   Max                27516 non-null  float64
 4   Median             27516 non-null  float64
 5   Mean               27516 non-null  float64
 6   SD                 27516 non-null  float64
dtypes: float64(7)
mem

In [ ]:
features["Label"] = features["Label"].astype(int)
pd.options.display.float_format = "{:,.2f}".format
print(features.info())
print(features.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27516 entries, 0 to 27515
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Label              27516 non-null  int64  
 1   Number of Samples  27516 non-null  float64
 2   Min                27516 non-null  float64
 3   Max                27516 non-null  float64
 4   Median             27516 non-null  float64
 5   Mean               27516 non-null  float64
 6   SD                 27516 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB
None
   Label  Number of Samples    Min    Max  Median   Mean    SD
0      1              79.00 305.00 520.00  375.00 378.54 30.51
1      1              83.00 285.00 390.00  365.00 360.66 16.91
2      1              83.00 200.00 525.00  360.00 363.01 30.62
3      1              84.00 335.00 365.00  355.00 356.67  5.63
4      1              85.00 340.00 365.00  355.00 354.59  5.04


In [ ]:
features["Label"].value_counts()

1    13758
0    13758
Name: Label, dtype: int64

In [ ]:
cols = ["Min", "Max", "Median", "Mean", "SD"]
X = features[cols]
y = features["Label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=None)
print("x train: ",X_train.shape)
print("x test: ",X_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

x train:  (16509, 5)
x test:  (11007, 5)
y train:  (16509,)
y test:  (11007,)


In [ ]:
print(X.shape)
print(y.shape)

(27516, 5)
(27516,)


Models


1. Logistic Regression using cross validation

In [ ]:
#with hyperparameter tuning
LR = LogisticRegression()

#Search grid for optimal parameters
lr_param_grid = {"penalty": ["l1", "l2", "elasticnet", "none"],
                 "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
                 "C": [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 1.0]}

gsLR = GridSearchCV(LR, param_grid = lr_param_grid, cv=10, scoring='accuracy', n_jobs= -1, verbose = 1)

gsLR.fit(X,y)

lr_best = gsLR.best_estimator_

Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 329 tasks      | elapsed:   52.9s
[Parallel(n_jobs=-1)]: Done 863 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed:  4.3min finished


In [ ]:
print(lr_best)
print(gsLR.best_score_)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
0.9306237081864523


In [ ]:
scoring = {'accuracy': 'accuracy', 'precision': 'precision', 'recall': 'recall', 'auc': 'roc_auc'}

modelCV = LogisticRegression(C= 1.0, penalty='l1', solver='liblinear')

results = cross_validate(modelCV, X, y, cv=5, scoring=list(scoring.values()), 
                         return_train_score=False)

print('K-fold cross-validation results:')
for sc in range(len(scoring)):
    print(modelCV.__class__.__name__+" %s: %.3f (+/-%.3f)" % (list(scoring.keys())[sc], results['test_%s' % list(scoring.values())[sc]].mean(), 
                                                              results['test_%s' % list(scoring.values())[sc]].std()))

# y_pred = gsLR.predict(X_test)

# print("\n GridSearch Results:")
# print("accuracy is %2.3f" % accuracy_score(y_test, y_pred))
# print("precision is %2.3f" % precision_score(y_test, y_pred))
# print("recall is %2.3f" % recall_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


K-fold cross-validation results:
LogisticRegression accuracy: 0.930 (+/-0.052)
LogisticRegression precision: 0.929 (+/-0.040)
LogisticRegression recall: 0.933 (+/-0.101)
LogisticRegression auc: 0.963 (+/-0.039)


2. Decision Trees with AdaBoost

In [ ]:
#with hyperparameter tuning
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

#Search grid for optimal parameters
ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[10,20,30,40,50],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 1.5]}

gsadaDTC = GridSearchCV(adaDTC, param_grid = ada_param_grid, cv=10, scoring='accuracy', n_jobs= -1, verbose = 1)

gsadaDTC.fit(X,y)

ada_best = gsadaDTC.best_estimator_

Fitting 10 folds for each of 280 candidates, totalling 2800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 982 tasks      | elapsed:   59.4s
[Parallel(n_jobs=-1)]: Done 2146 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2800 out of 2800 | elapsed:  2.4min finished


In [ ]:
print(ada_best)
print(gsadaDTC.best_score_)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                      

In [ ]:
scoring = {'accuracy': 'accuracy', 'precision': 'precision', 'recall': 'recall', 'auc': 'roc_auc'}

modelCV = AdaBoostClassifier(DecisionTreeClassifier(criterion="entropy", splitter='random'))

results = cross_validate(modelCV, X, y, cv=5, scoring=list(scoring.values()), 
                         return_train_score=False)

print('K-fold cross-validation results:')
for sc in range(len(scoring)):
    print(modelCV.__class__.__name__+" %s: %.3f (+/-%.3f)" % (list(scoring.keys())[sc], results['test_%s' % list(scoring.values())[sc]].mean(), 
                                                              results['test_%s' % list(scoring.values())[sc]].std()))
    
# y_pred = gsadaDTC.predict(X_test)

# print("accuracy is %2.3f" % accuracy_score(y_test, y_pred))
# print("precision is %2.3f" % precision_score(y_test, y_pred))
# print("recall is %2.3f" % recall_score(y_test, y_pred))

K-fold cross-validation results:
AdaBoostClassifier accuracy: 0.932 (+/-0.029)
AdaBoostClassifier precision: 0.939 (+/-0.049)
AdaBoostClassifier recall: 0.927 (+/-0.045)
AdaBoostClassifier auc: 0.932 (+/-0.029)


3. Random Forest

In [ ]:
#with hyperparameter tuning
RFC = RandomForestClassifier()

#Search grid for optimal parameters
rf_param_grid = {"max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "n_estimators" :[100,200,300],
              "criterion": ["gini", "entropy"]}

gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=5, scoring="accuracy", n_jobs= -1, verbose = 1)

gsRFC.fit(X,y)

rfc_best = gsRFC.best_estimator_

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed: 32.1min finished


In [ ]:
print(rfc_best)
print(gsRFC.best_score_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
0.9519565173753005


In [ ]:
scoring = {'accuracy': 'accuracy', 'precision': 'precision', 'recall': 'recall', 'auc': 'roc_auc'}

modelCV = RandomForestClassifier(criterion="gini", min_samples_split=2, min_samples_leaf=10, max_features=3, n_estimators=10)

results = cross_validate(modelCV, X, y, cv=5, scoring=list(scoring.values()), 
                         return_train_score=False)

print('K-fold cross-validation results:')
for sc in range(len(scoring)):
    print(modelCV.__class__.__name__+" %s: %.3f (+/-%.3f)" % (list(scoring.keys())[sc], results['test_%s' % list(scoring.values())[sc]].mean(), 
                                                              results['test_%s' % list(scoring.values())[sc]].std()))

# y_pred = gsRFC.predict(X_test)

# print("accuracy is %2.3f" % accuracy_score(y_test, y_pred))
# print("precision is %2.3f" % precision_score(y_test, y_pred))
# print("recall is %2.3f" % recall_score(y_test, y_pred))

K-fold cross-validation results:
RandomForestClassifier accuracy: 0.950 (+/-0.028)
RandomForestClassifier precision: 0.949 (+/-0.042)
RandomForestClassifier recall: 0.956 (+/-0.059)
RandomForestClassifier auc: 0.975 (+/-0.024)


4. Artificial Neural Network

In [ ]:
#defining the model
model = Sequential()
model.add(Dense(12, input_dim=5, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

# fit the keras model on the dataset
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=128)

Epoch 1/200
129/129 [==============================] - 2s 7ms/step - loss: 3.1759 - accuracy: 0.8598 - precision_1: 0.9234 - recall_1: 0.7853 - auc_1: 0.8795 - val_loss: 0.9840 - val_accuracy: 0.9006 - val_precision_1: 0.9215 - val_recall_1: 0.8743 - val_auc_1: 0.9576
Epoch 2/200
129/129 [==============================] - 0s 3ms/step - loss: 0.7612 - accuracy: 0.9014 - precision_1: 0.9064 - recall_1: 0.8983 - auc_1: 0.9488 - val_loss: 0.3503 - val_accuracy: 0.9126 - val_precision_1: 0.8938 - val_recall_1: 0.9350 - val_auc_1: 0.9504
Epoch 3/200
129/129 [==============================] - 0s 3ms/step - loss: 0.3228 - accuracy: 0.9101 - precision_1: 0.9054 - recall_1: 0.9154 - auc_1: 0.9517 - val_loss: 0.2717 - val_accuracy: 0.9192 - val_precision_1: 0.9223 - val_recall_1: 0.9144 - val_auc_1: 0.9542
Epoch 4/200
129/129 [==============================] - 0s 3ms/step - loss: 0.2759 - accuracy: 0.9103 - precision_1: 0.9084 - recall_1: 0.9152 - auc_1: 0.9527 - val_loss: 0.2575 - val_accuracy: 

In [ ]:
# evaluate the keras model
_, train_acc, train_pre, train_rec, train_auc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc, test_pre, test_rec, test_auc = model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
print('Train: %.3f, Test: %.3f' % (train_pre, test_pre))
print('Train: %.3f, Test: %.3f' % (train_rec, test_rec))
print('Train: %.3f, Test: %.3f' % (train_auc, test_auc))

Train: 0.949, Test: 0.953
Train: 0.934, Test: 0.938
Train: 0.967, Test: 0.971
Train: 0.986, Test: 0.987


5. Stacking

Using Alogrithms like DT, RF, Gradient Boosting, Extra Trees Classifier

In [ ]:
# Put in our parameters for said classifiers
# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

In [ ]:
def param(clf, param):
  return clf(**param)

In [ ]:
# Create 5 objects that represent our 4 models
lr = LogisticRegression(C= 1.0, penalty='l2')
rf = RandomForestClassifier(criterion="gini", min_samples_split=2, min_samples_leaf=10, max_features=3, n_estimators=10)
et = param(ExtraTreesClassifier, et_params)
ada = AdaBoostClassifier(DecisionTreeClassifier(criterion="entropy", splitter='random'))
gb = param(GradientBoostingClassifier, gb_params)

#combine all into one model
models = [lr, rf, et, ada, gb]

In [ ]:
# Create our OOF train and test predictions
L1_train, L1_test = stacking(models, X_train, y_train, X_test, regression=False, n_folds=5)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
print(L1_train.shape)
print(L1_test.shape)

(16509, 5)
(11007, 5)


Using XGBoostClassifier as meta_model

In [ ]:
gbm = xgb.XGBClassifier(
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1)

gbm.fit(L1_train, y_train)
y_pred = gbm.predict(L1_test)

score = accuracy_score(y_test, y_pred)
score1 = precision_score(y_test, y_pred)
score2 = recall_score(y_test, y_pred)
score3 = roc_auc_score(y_test, y_pred)
print(score)
print(score1)
print(score2)
print(score3)

0.9720178068501862
0.9566141313972021
0.9884720951509607
0.9721321139775013
